# Praktikum Pengantar Pembelajaran Mesin


---
## Bab 7. Support Vector Machine (SVM) Lanjutan


### 1) Import Data

Unduh dataset yang akan digunakan pada praktikum kali ini. Anda dapat menggunakan aplikasi wget untuk mendowload dataset dan menyimpannya dalam Google Colab. Jalankan cell di bawah ini untuk mengunduh dataset

In [ ]:
!wget https://gist.githubusercontent.com/netj/8836201/raw/6f9306ad21398ea43cba4f7d537619d0e07d5ae3/iris.csv

--2023-04-26 15:12:55--  https://gist.githubusercontent.com/netj/8836201/raw/6f9306ad21398ea43cba4f7d537619d0e07d5ae3/iris.csv
Resolving gist.githubusercontent.com (gist.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to gist.githubusercontent.com (gist.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3975 (3.9K) [text/plain]
Saving to: ‘iris.csv’

iris.csv            100%[===================>]   3.88K  --.-KB/s    in 0s      

2023-04-26 15:12:55 (46.8 MB/s) - ‘iris.csv’ saved [3975/3975]



Setelah dataset berhasil diunduh, langkah berikutnya adalah membaca dataset dengan memanfaatkan fungsi **readcsv** dari library pandas. Lakukan pembacaan berkas csv ke dalam dataframe dengan nama **data** menggunakan fungsi **readcsv**. Jangan lupa untuk melakukan import library pandas terlebih dahulu


In [ ]:
import pandas as pd
import numpy as np
data = pd.read_csv('iris.csv')



Cek isi dataset Anda dengan menggunakan perintah **head()**

In [ ]:
data.head()

,sepal.length,sepal.width,petal.length,petal.width,variety
0,5.1,3.5,1.4,0.2,Setosa
1,4.9,3.0,1.4,0.2,Setosa
2,4.7,3.2,1.3,0.2,Setosa
3,4.6,3.1,1.5,0.2,Setosa
4,5.0,3.6,1.4,0.2,Setosa


## 2) Membagi data menjadi data latih dan data uji

Metode pembelajaran mesin memerlukan dua jenis data :


1.   Data latih : Digunakan untuk proses training metode klasifikasi
2.   Data uji : Digunakan untuk proses evaluasi metode klasifikasi

Data uji dan data latih perlu dibuat terpisah (mutualy exclusive) agar hasil evaluasi lebih akurat.

Data uji dan data latih dapat dibuat dengan cara membagi dataset dengan rasio tertentu, misalnya 80% data latih dan 20% data uji.

Library Scikit-learn memiliki fungsi [train_test_split](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.train_test_split.html) pada modul **model_selection** untuk membagi dataset menjadi data latih dan data uji. Bagilah dataset anda menjadi dua, yaitu **data_latih** dan **data_uji**.


In [ ]:
from sklearn.model_selection import train_test_split
data_latih, data_uji = train_test_split(data,test_size=0.2)

Tampilkan banyaknya data pada **data_latih** dan **data_uji**. Seharusnya **data_latih** terdiri dari 120 data, dan **data_uji** terdiri dari 30 data

In [ ]:
print(data_uji.shape[0])
print(data_latih.shape[0])

30
120


Pisahkan label/kelas dari data uji menjadi sebuah variabel bernama **label_uji**

In [ ]:
label_uji = data_uji.pop('variety')

## 3) Pembentukan data latih one-vs-rest

Metode one-vs-rest memerlukan tiga jenis data latih yang diperlukan untuk melatih tiga SVM yang berbeda pada dataset Iris. Fungsi **buat_trainingset** digunakan untuk membentuk tiga dataset tersebut.

In [ ]:
def buat_trainingset(dataset):
  trainingset = {}
  kolom_kelas = dataset.columns[-1]
  list_kelas = dataset[kolom_kelas].unique()
  for kelas in list_kelas:
    data_temp = dataset.copy(deep=True)
    data_temp[kolom_kelas]=data_temp[kolom_kelas].map({kelas:1})
    data_temp[kolom_kelas]=data_temp[kolom_kelas].fillna(-1)
    trainingset[kelas]=data_temp
  return trainingset

Gunakan fungsi **buat_trainingset** untuk membentuk data latih dengan nama variabel **trainingset** yang akan digunakan pada proses training. 

In [ ]:
trainingset = buat_trainingset(data_latih)

Tampilkan isi **trainingset** agar Anda dapat memahami struktur dari variabel tersebut.

In [ ]:
print(trainingset)

{'Setosa':      sepal.length  sepal.width  petal.length  petal.width  variety
23            5.1          3.3           1.7          0.5      1.0
36            5.5          3.5           1.3          0.2      1.0
61            5.9          3.0           4.2          1.5     -1.0
80            5.5          2.4           3.8          1.1     -1.0
29            4.7          3.2           1.6          0.2      1.0
..            ...          ...           ...          ...      ...
4             5.0          3.6           1.4          0.2      1.0
24            4.8          3.4           1.9          0.2      1.0
10            5.4          3.7           1.5          0.2      1.0
139           6.9          3.1           5.4          2.1     -1.0
70            5.9          3.2           4.8          1.8     -1.0

[120 rows x 5 columns], 'Versicolor':      sepal.length  sepal.width  petal.length  petal.width  variety
23            5.1          3.3           1.7          0.5     -1.0
36          

## 4) Pembentukan SVM Biner

Tujuan dari algoritma SVM adalah meminimalkan nilai *cost function*. Penghitungan nilai minimal dapat dapat dilakukan dengan menghitung nilai gradien dari *cost function* terlebih dahulu. Fungsi di bawah ini berguna untuk menghitung nilai gradien cost function

In [ ]:
def hitung_cost_gradient(W,X,Y,regularization):
  jarak = 1 - (Y*np.dot(X,W))
  dw = np.zeros(len(W))
  if max(0,jarak)==0:
    di=W
  else:
    di = W - (regularization * Y*X)
  dw += di
  return dw

Terdapat beberapa cara untuk meminimalkan nilai *cost function*, salah satunya menggunakan Stochastic Gradient Descent (SGD) untuk melakukan minimasi. Minimasi *cost function* merupakan inti dari algoritma SVM. Fungsi di bawah ini merupakan implementasi algoritma SGD 

In [ ]:
from sklearn.utils import shuffle
def sgd(data_latih,label_latih,learning_rate = 0.000001,max_epoch=1000,regularization=10000):
  data_latih = data_latih.to_numpy()
  label_latih = label_latih.to_numpy()
  bobot = np.zeros(data_latih.shape[1])
  for epoch in range(1,max_epoch):
    X,Y =shuffle(data_latih,label_latih,random_state=101)
    for index,x in enumerate(X):
      delta = hitung_cost_gradient(bobot,x,Y[index],regularization)
      bobot = bobot - (learning_rate * delta)
  return bobot

## 5) Proses Training

Proses training dilakukan dengan memanggil fungsi **sgd** berulang kali sesuai banyaknya kelas yang ada pada data. Dengan demikian, proses training menghasilkan bobot sebanyak kelas yang ada pada dataset. Buatlah fungsi bernama **training** yang digunakan untuk melakukan proses training one-vs-rest

In [ ]:
def training(trainingset):
  list_kelas = trainingset.keys()
  w= {}
  for kelas in list_kelas:
    data_latih = trainingset[kelas]
    label_latih = data_latih.pop(data_latih.columns[-1])
    w[kelas] = sgd(data_latih,label_latih)
  return w

Lakukan proses training dengan memanggil fungsi **training** dan menempatkan hasilnya pada variabel **W**

In [ ]:
W = training(trainingset)

Tampilkan isi variabel **W**

In [ ]:
print(W)

{'Setosa': array([ 0.152874  ,  0.72686667, -1.02249322, -0.52975148]), 'Versicolor': array([ 1.22590297, -2.48354411,  0.73350702, -2.97676379]), 'Virginica': array([-3.62206283, -2.56915366,  3.99244013,  6.15243503])}


## 6) Proses *testing* biner
Proses testing dilakukan dengan menghitung nilai [*dot product*](https://en.wikipedia.org/wiki/Dot_product) antara bobot hasil training dengan data uji. Kelas data ditentukan berdasarkan tanda (positif atau negatif) dari hasil dot product tersebut. Fungsi berikut mengimplementasikan proses testing

In [ ]:
def testing(W,data_uji):
  prediksi = np.array([])
  for i in range(data_uji.shape[0]):
    y_prediksi = np.sign(np.dot(W,data_uji.to_numpy()[i]))
    prediksi = np.append(prediksi,y_prediksi)
  return prediksi

In [ ]:
y_prediksi = testing(W['Versicolor'], data_uji)
temp = label_uji.copy(deep=True)
x_label=temp.map({'Versicolor':1})
x_label=x_label.fillna(-1)
print(sum(y_prediksi==x_label))

19


## TUGAS
Pada tugas kali ini Anda mendefinisikan proses testing pada metode one-vs-rest. Proses testing pada metode one-vs-rest dilakukan dengan memanggil proses testing biner untuk setiap **value** pada dictionary **W**. Kelas pada sebuah data latih adalah **key** pada dictionary **W** yang memiliki nilai prediksi **1**. Lengkapi fungsi **testing_onevsrest** di bawah ini. Output dari fungsi tersebut adalah list nama kelas hasil prediksi.

In [ ]:
import random
def testing_onevsrest(W,data_uji):
  prediksi = {}
  kelas_prediksi = np.array(['kelas_kosong']*data_uji.shape[0])
  list_kelas = W.keys()
  for kelas in list_kelas:
    prediksi[kelas] = testing(W[kelas],data_uji)
    prediksi[kelas] = np.where(prediksi[kelas]==1,kelas,prediksi[kelas])
    idx = np.where(prediksi[kelas]==kelas)
    for i in idx:
      kelas_prediksi[i] = kelas
  list_k = list(list_kelas)
  kelas_prediksi = np.where(kelas_prediksi=='kelas_kosong',random.choice(list_k),kelas_prediksi)
  return kelas_prediksi

In [ ]:
prediksi = testing_onevsrest(W,data_uji)

Berapa banyak data latih yang berhasil diprediksi dengan benar?

In [ ]:
print(prediksi)

['Virginica' 'Virginica' 'Setosa' 'Virginica' 'Versicolor' 'Setosa'
 'Versicolor' 'Virginica' 'Setosa' 'Virginica' 'Setosa' 'Versicolor'
 'Virginica' 'Setosa' 'Setosa' 'Virginica' 'Virginica' 'Setosa'
 'Virginica' 'Setosa' 'Virginica' 'Setosa' 'Setosa' 'Versicolor'
 'Virginica' 'Setosa' 'Setosa' 'Virginica' 'Setosa' 'Setosa']


In [ ]:
#prediksi dengan data uji
print('Jumlah prediksi benar = ',sum(prediksi==label_uji))

Jumlah prediksi benar =  21


In [ ]:
#prediksi dengan data latih
prediksi_latih = testing_onevsrest(W,data_latih.drop(['variety'],axis=1))
print('Jumlah prediksi benar = ',sum(prediksi_latih==data_latih['variety']))

Jumlah prediksi benar =  92


Kesimpulannya adalah: Terdapat 24 data yang diprediksi dengan benar dari 30 data uji. Terdapat 117 data yang diprediksi dengan benar dari 120 data latih.